In [2]:
import os

os.environ['JAVA_HOME'] = "C:/Program Files/Java/jdk-11"
os.environ['PYSPARK_PYTHON'] = "C:/Users/usr/anaconda3/envs/pyspark_env/python.exe"
os.environ['PYSPARK_DRIVER_PYTHON'] = "C:/Users/usr/anaconda3/envs/pyspark_env/python.exe"
os.environ['HADOOP_HOME'] = "C:/hadoop-3.4.0"
os.environ['HADOOP_COMMON_LIB_NATIVE_DIR'] = "C:/hadoop-3.4.0/lib/native"
os.environ['PATH'] += os.pathsep + "C:/hadoop-3.4.0/bin"

import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

sc = spark.sparkContext

### **Almacenamiento en caché**

- persist() y cache()

- cache() = persist(MEMORY_ONLY)

In [3]:
rdd = sc.parallelize([item for item in range(10)])

In [4]:
from pyspark.storagelevel import StorageLevel

In [5]:
rdd.persist(StorageLevel.MEMORY_ONLY) # Persiste nuestro RDD solo a nivel de memoria.

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:287

- Si queremos cambiarle el nivel de persistencia a este rdd tenemos antes que hace **unpersist()**

In [6]:
rdd.unpersist()

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:287

In [7]:
rdd.persist(StorageLevel.DISK_ONLY) # Persiste nuestro RDD solo a nivel de disco.

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:287

In [8]:
rdd.unpersist()

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:287

In [9]:
rdd.cache() # Nivel de persistencia solo en memoria.

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:287

### **Particionado y mezcla de datos(Shuffling)**

<img src="https://github.com/adrianlardies/lab-pyspark/raw/main/images/Shuffling.PNG" width="700"/>

<img src="https://github.com/adrianlardies/lab-pyspark/raw/main/images/Shuffling_2.PNG" width="700"/>

- #### **HashPartitioner**

In [10]:
rdd1 = sc.parallelize(['x', 'y', 'z'])

In [11]:
hola = 'Hola'

In [12]:
hash(hola)

4710454791444069950

In [13]:
num_particiones = 6

- indice = hash(item) %(dividido) num_particiones

- indice: hacía qué partición debe ir cada uno de los elementos del RDD cuando Spark realiza el particionado.

In [ ]:
hash('x') % num_particiones # El valor 'x' iria a la partición 3

3

In [ ]:
hash('y') % num_particiones # El valor 'y' iria a la partición 3

3

In [ ]:
hash('z') % num_particiones # El valor 'z' iria a la partición 4

4

<img src="https://github.com/adrianlardies/lab-pyspark/raw/main/images/HashPartitioner.PNG" width="700"/>

- #### **RangePartitioner**

- Divide el RDD en rangos aproximadamente iguales.

- Primero debemos tener el RDD para establecer límites razonables a las particiones.

- #### **Mezcla de datos (Shuffling)**

<img src="https://github.com/adrianlardies/lab-pyspark/raw/main/images/Shuffling_3.PNG" width="700"/>

### **Broadcast de variables**

- Variables compartidas entre todos los ejecutores.

- Se crean una vez en el controlador y se leen en los ejecutores.

In [19]:
rdd2 = sc.parallelize([item for item in range(10)])

In [24]:
rdd2.collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [20]:
uno = 1

In [21]:
br_uno = sc.broadcast(uno) # br = variable. El valor 'uno' es compartido entre todas las particiones.

In [22]:
rdd3 = rdd2.map(lambda x: x + br_uno.value) # br.value = valor compartido. br.value = 1

In [23]:
rdd3.collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

- La variable broadcast ocupa memoria en todos los ejecutores.

- Según el tamaño puede causar problema en la memoria de los ejecutores.

- Podemos eliminar la variable de la memoria de todos los ejecutores.

In [25]:
br_uno.unpersist()

- Si la vuelvo a llamar se vuelve a almacenar en memoria de todos los ejecutores.

In [26]:
rdd3 = rdd2.map(lambda x: x + br_uno.value)

In [27]:
rdd3.collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

- Se puede eliminar definitivamente la variable de todos los ejecutores y del controlador también.

In [28]:
br_uno.destroy()

### **Acumuladores**

- Son variables compartidas entre ejecutores que normalmente se utilizan para agregar contadores a su programa Spark.

In [29]:
acumulador = sc.accumulator(0) # 0 es el valor inicial del acumulador.

In [30]:
rdd4 = sc.parallelize([2, 4, 6, 8, 10])

In [31]:
rdd4.foreach(lambda x: acumulador.add(x)) # Suma a nuestro acumulador cada uno de los elementos del RDD.

In [ ]:
acumulador.value # Suma de los elementos del RDD.

30

In [33]:
rdd5 = sc.parallelize('Mi nombre es Adrián Lardiés y me siento genial.'.split(' '))

In [34]:
acumulador1 = sc.accumulator(0)

In [35]:
rdd5.foreach(lambda x: acumulador1.add(1)) # Queremos que cuente los elementos, que sume 1 cada vez que itera por los elementos del RDD.

In [36]:
acumulador1.value

9